# Video Content Moderation

Video content moderation is a common use case across various industries, including Social Media, Media & Entertainment, Advertising, Education and more. Customers need to prevent unsafe or toxic content that could damage brand safety, or violate regional regulations. 

BDA Video offers features that help customers detect unsafe content in both the visual and audio components of videos. With blueprints, customers can extend their moderation policies to flexibly support their business needs.

In this lab, we will use BDA to analyze a sample video containing unsafe and toxic content:

![video moderation](../static/bda-video-cm.png)

## Prerequisits

In [ ]:
%pip install "boto3>=1.37.4" "moviepy==2.1.2" --upgrade -qq

In [ ]:
import boto3
import json
import uuid
import utils
import sagemaker

bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

For a self-hosted workshop, we recommend creating a new S3 bucket in the same region where you plan to run the workshop. You can name it `bda-workshop-YOUR_ACCOUNT_ID-YOUR_REGION`.

In [ ]:
data_bucket = sagemaker.Session().default_bucket() # Enter your bucket name here
data_prefix = "bda-workshop/image"
output_prefix = "bda-workshop/image/ouput"

print('Workshop S3 bucket:', data_bucket)

In [ ]:
# Get current AWS account Id and region
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
region = boto3.Session().region_name

print(f'Current AWS account Id: {account_id}, region name: {region}')

## Create a BDA project
To start a BDA job, you need a BDA project, which organizes both standard and custom output configurations. This project is reusable, allowing you to apply the same configuration to process multiple videos that share the same settings.

In the code snippet below, we create a BDA project with standard output configurations for video modality. These configurations can be tailored to extract only the specific information you need. In this lab, we will enable the below video outputs:
- Full video summary
- Content moderation (visual and audio)
- Text in video (without bounding-boxes)
- Logo in video (without bounding-boxes)

For a complete API reference for creating a BDA project, refer to this [document](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation/client/create_data_automation_project.html).

In [ ]:
response = bda_client.create_data_automation_project(
    projectName=f'bda-workshop-video-project-moderation-{str(uuid.uuid4())[0:4]}',
    projectDescription='BDA workshop video sample project - content moderation',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['CONTENT_MODERATION','TEXT_DETECTION','TRANSCRIPT']
                },
                'boundingBox': {
                    'state': 'ENABLED'
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': [
                    'VIDEO_SUMMARY',
                ]
            }
        }
    }
)

The `create_data_automation_project` API will return the project ARN, which we will use it to invoke the video analysis task.

In [ ]:
video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

## Start an asynchronous BDA task to extract and analyze a video
In this section, we will use a sample video contains unsafe content, and extract and analyze it using BDA, applying the configuration defined in the BDA project. 

### Prepare the sample video

In [ ]:
# Download sample video
sample_video_path = 'content-moderation-demo.mp4'
source_url = 'https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/335119c4-e170-43ad-b55c-76fa6bc33719/content-moderation-demo.mp4'

!curl {source_url} --output {sample_video_path}

Let's display the video. This short sample contains unsafe visual chapters, including alcohol, tobacco, and suggestive content, as well as toxic audio components. It also features text in the visual portion, which is considered unsafe. Our mission is to extract and analyze this information for content moderation purposes.

In [ ]:
from IPython.display import Video
Video(sample_video_path, width=800)

To analyze the video using BDA, we need to upload it to an S3 bucket that BDA can access. 

In [ ]:
s3_key = f'{data_prefix}/{sample_video_path.split("/")[-1]}'
s3_client.upload_file(sample_video_path, data_bucket, s3_key)

### Start BDA task
We will now invoke the BDA API to process the uploaded video. You need to provide the BDA project ARN that we created at the beginning of the lab and specify an S3 location where BDA will store the output results.

For a complete API reference for invoke a BDA async task, refer to this [document](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation-runtime/client/invoke_data_automation_async.html).

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{data_bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{data_bucket}/{output_prefix}'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=f'arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1'
)

The `invoke_data_automation_async` API is asynchronous. It returns an invocation task identifier, `invocationArn`. We can then use another API `get_data_automation_status` to monitor the task's status until it completes.

> In production workloads, an event-driven pattern is recommended. Allow BDA to trigger the next step once the task is complete. This can be achieved by configuring the notificationConfiguration in the invoke task, which will send a notification to a subscribed AWS service, such as a Lambda function. Alternatively, you can set up an S3 trigger on the bucket where BDA will drop the results.

In [ ]:
invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

In this lab, we will use the loop below to monitor the task by calling the `get_data_automation_status` API every 5 seconds until the task is complete.

This video will take less than 5 minutes to process.

In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

## Access the BDA analysis result
The `get_data_automation_status` API returns an S3 URI containing the result configuration, which provides the S3 location where BDA outputs the extraction results. We will then parse this file to retrieve the result path.

In [ ]:
config_data = utils.read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

As shown above, the BDA output configuration file contains metadata about the BDA result, including the job ID, status, modality, and the S3 location of the actual result JSON. We will now download this result file to verify the output.

In [ ]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = utils.read_json_on_s3(result_uri,s3_client)

JSON(result_data)

## Review the result
The BDA video analysis results provide a detailed breakdown of information, organized by video and chapter levels. 
> A video chapter is a sequence of shots that form a coherent unit of action or narrative within the video.

In this lab, we are using a short sample video that contains only one chapter, as our focus will be on reviewing the moderation analysis output. For more detailed instructions on metadata on chapter and frame level, such as summary, IAB, and bounding boxes, please refer to the previous lab.

### Full video transcription
At the video level, we receive the full transcript based on the video's audio, with speakers identified.

In [ ]:
print(result_data["video"]["transcript"]["representation"]["text"])

### Audio transcripts with toxicity analysis
You can find two fields at the chapter level containing audio transcripts and audio moderation data: `audio_segments` and `content_moderation`. The sequence of moderation (toxicity) analysis corresponds to the transcripts. Below, we display them side by side.

In [ ]:
import matplotlib.pyplot as plt

for chapter in result_data["chapters"]:
    for idx in range(0,len(chapter["audio_segments"])):
        print(f'[{chapter["audio_segments"][idx]["start_timestamp_millis"]/1000} - {chapter["audio_segments"][idx]["end_timestamp_millis"]/1000}] {chapter["audio_segments"][idx]["text"]}')
        
        data = chapter["content_moderation"][idx]
        # Extract category names and their confidence values
        categories = [item["category"] for item in data["moderation_categories"]]
        confidence_values = [item["confidence"] for item in data["moderation_categories"]]
        
        # Plotting
        plt.figure(figsize=(10, 6))
        plt.bar(categories, confidence_values, color='skyblue')
        
        # Add title and labels
        plt.title(f'Moderation Confidence by Category - overall confidence {data["confidence"]*100}%')
        plt.xlabel('Moderation Categories')
        plt.ylabel('Confidence')
        plt.xticks(rotation=45, ha='right')
        
        # Display the plot
        plt.tight_layout()
        plt.show()

The bar chart above represents the audio segment with its start and end timestamps. It visualizes the toxicity analysis of the audio transcript, where the x-axis shows the toxicity categories and the y-axis displays the corresponding confidence scores. 

BDA video toxicity analysis support the below built-in toxicity categories with confidence score between 0 and 1:
- profanity
- hate_speech
- sexual
- insult
- violence_or_threat
- graphic
- harassment_or_abuse

### Visual content moderation the frame level with confidence score
BDA also analyzes the visual portion of the video and provides content moderation labels at the frame level. You can find the visual moderation output for each chapter under the respective frames.

In [ ]:
utils.plot_content_moderation(sample_video_path, result_data, 0)

### Frame level text extraction and confidence scores
Text extraction, along with confidence scores, is available at the frame level. In the output JSON structure, frames are organized under each chapter with defined start and end times. If text is detected at a given frame timestamp, you can find `text_words` and `text_lines` included at the frame level.

Let's plot the frames with detected text.

In [ ]:
utils.plot_text(sample_video_path, result_data, 0)

### Video blueprint for customized policy evaluation - coming soon

## Summary
In this lab, we use BDA to extract and analyze a sample video containing unsafe and toxic content. The BDA output includes visual and audio moderation detections, as specified in the BDA project configuration.

## Clean up

Delete the BDA project, blueprint, image, and result from S3.

In [ ]:
# delete BDA project
response = bda_client.delete_data_automation_project(
    projectArn=video_project_arn
)
response

In [ ]:
# delete uploaded image from S3
s3_client.delete_object(Bucket=data_bucket, Key=s3_key)

In [ ]:
# delete results from S3
utils.delete_s3_folder(data_bucket, output_config.replace("job_metadata.json","") ,s3_client)